In [17]:
# !pip install cellrank
# !pip install scanpy
# !pip install scipy
# !pip install anndata


In [2]:
%%bash

source downloadData.sh

!mkdir -p ./data/GSE107185

# SUESS dataset
download_dataset "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE107185&format=file" "./data" "GSE107185_RAW.tar"
tar -xvf ./data/GSE107185_RAW.tar -C ./data/GSE107185

# for all files in ./data/ unzip them

# Check if the target directory was provided
if [[ -z "./data" ]]; then
    echo "Usage: $0 <target-directory>"
    exit 1
fi

# Navigate to the target directory
cd "./data" || exit

# Find and decompress all .gz files in the directory
for file in *.gz; do
    if [[ -f "$file" ]]; then
        echo "Decompressing $file"
        gunzip "$file"
    else
        echo "No .gz files found in $TARGET_DIR"
        break
    fi
done



# look into the single cell dataset. 
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE213462


# each of the cells has one barcode and one TF, need use the dictionary to get the TF to the gene counts. 

# the rows are the genes the columns are the cells.

# the files are different types of differntiation procedures (we think) 
# read the paper.


bash: line 4: !mkdir: command not found


The file ./data/GSE107185_RAW.tar already exists. Skipping download.


tar: data/GSE107185: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Decompressing GSE154759_adata.h5ad.gz


gzip: GSE154759_adata.h5ad already exists;	not overwritten


...

Decompressing GSE217460_210322_TFAtlas_differentiated.h5ad.gz


gzip: GSE217460_210322_TFAtlas_differentiated.h5ad already exists;	not overwritten


Decompressing GSE217460_210322_TFAtlas_differentiated_raw.h5ad.gz


gzip: GSE217460_210322_TFAtlas_differentiated_raw.h5ad already exists;	not overwritten


Decompressing GSM3308844_up-tf-stem-sample2.counts.tsv.gz
Decompressing GSM3308844_up-tf-stem-sample2_pheno_dict.csv.gz
Decompressing GSM3308845_up-tf-stem-sample3.counts.tsv.gz
Decompressing GSM3308845_up-tf-stem-sample3_pheno_dict.csv.gz
Decompressing GSM3308846_up-tf-endo-sample1.counts.tsv.gz
Decompressing GSM3308846_up-tf-endo-sample1_pheno_dict.csv.gz
Decompressing GSM3308847_up-tf-endo-sample2.counts.tsv.gz
Decompressing GSM3308847_up-tf-endo-sample2_pheno_dict.csv.gz
Decompressing GSM3308848_up-tf-multi-sample1.counts.tsv.gz
Decompressing GSM3308848_up-tf-multi-sample1_pheno_dict.csv.gz
Decompressing GSM3308849_up-tf-multi-sample2.counts.tsv.gz
Decompressing GSM3308849_up-tf-multi-sample2_pheno_dict.csv.gz
Decompressing GSM3308850_up-tf-klf.counts.tsv.gz
Decompressing GSM3308850_up-tf-klf-myc_pheno_dict.csv.gz
Decompressing GSM3308850_up-tf-myc.counts.tsv.gz
Decompressing GSM3308851_up-tf-neuron-hygro-sample1.counts.tsv.gz
Decompressing GSM3308851_up-tf-neuron-hygro-sample1_phe

In [1]:
import cellrank as  cr
import scanpy as sc
from scipy import sparse
import pandas as pd
import numpy as np
from anndata import AnnData

 Transcript abundance (TPM) measurements for induced pluripotent stem cells (iPSCs), COV434 ovarian cancer cells, and iPSC-derived granulosa-like cells. 2. Differential gene expression analysis to identify effects of transcription factor overexpression during iPSC differentiation.

In [46]:
filenames = [ "GSM3308848_up-tf-multi-sample1.counts.tsv",
"GSM3308852_up-tf-neuron-hygro-sample2.counts.tsv",
"GSM3308844_up-tf-stem-sample2.counts.tsv",
"GSM3308849_up-tf-multi-sample2.counts.tsv",
"GSM3308853_up-tf-neuron-nohygro-sample1.counts.tsv",
"GSM3308845_up-tf-stem-sample3.counts.tsv",
"GSM3308850_up-tf-klf.counts.tsv",
"GSM3308854_up-tf-neuron-nohygro-sample2.counts.tsv",
"GSM3308846_up-tf-endo-sample1.counts.tsv",
"GSM3308850_up-tf-myc.counts.tsv",
"GSM3308847_up-tf-endo-sample2.counts.tsv",
"GSM3308851_up-tf-neuron-hygro-sample1.counts.tsv" ] 

dir = "./data"

filename2 = "./data/GSM3308852_up-tf-neuron-hygro-sample2_pheno_dict.csv"
# these are the transcription factors that are upregulated in the cells.
# corresponding with a bunch of the tags. "the TF barcode?"

# because we have |TF | TF-barcode | UMI | cell barcode | 
# but I think we only see the TF and TF-barcode(s)

# R1 is the barcode and R2 is biological. For R1, bases 1 - 16 correspond to the cell barcode and bases 17 -26 correspond to the unique molecular index (UMI).



def read_data(filename):
    filepath = f"{dir}/{filename}"
    df = pd.read_csv(filepath, delimiter='\t', header=0)
    # print(df.head())    
    # print(df.shape)
    # print(df.columns)
    # print(df.index)

    # obs = df.iloc[:, 0]  
    X = df.iloc[:, 1:].values  
    tdata = AnnData(X=X)
    # add filename to tdata
    # tdata.uns['filename'] = filename

    # relate the TAGS to the cells 
    return tdata
adata = read_data(filenames[1])

# treat other data as dictionary
# otherdata = pd.read_csv(filename2, delimiter=',', header=None, index_col=None)

import csv


# Initialize an empty dictionary
data_dict = {}

# Open the CSV file and read its contents
with open(filename2, mode='r', encoding='utf-8') as file:
    # Create a CSV reader
    reader = csv.reader(file)
    
    # Skip the header row if your CSV has a header
    next(reader, None)  # Uncomment this line if your CSV file has a header
    
    # Iterate over the CSV rows
    for row in reader:
        # Use the first column as the key and the second column (removing quotes) as the value
        key = row[0]
        value = row[1].strip('"')  # Strip double quotes from the value
        vallist = value.split(',')
        data_dict[key] = vallist

# Print the resulting dictionary
print(data_dict)


# adatas = []
# for filename in filenames:
#     tdata = read_data(filename)
#     print(tdata.obs.head())
#     tdata.obs['filename'] = filename
#     print(tdata.obs.head())
#     print(tdata.var.head())
#     print(tdata.X.shape)
#     print(tdata.X[:5, :5])
#     print(tdata.X[:5, :5].shape)
#     adatas.append(tdata)

# adata = adatas[0].concatenate(adatas[1:])
# # save
# !mkdir -p "./tempdata"
# adata.write_h5ad('./tempdata/GSE107185Anndata_concatinated.h5ad')


{'LHX3': ['TCAGATGCATTTCACT', 'TGAGGGAGTTCCGTCT', 'CCACTACGTTCAACCA', 'GGATTACCAGTTTACG', 'GTCACAATCGACAGCC', 'ACAGCTAGTGACAAAT', 'GTTCATTTCTAACTGG', 'TCAGCAAGTGCAACTT', 'GGATTACCACTATCTT', 'TCTCTAAAGAACAACT', 'GCATGTACACCATCCT', 'CGATTGAAGCAAATCA', 'CTACGTCTCTTGCCGT', 'GACCTGGTCAGCACAT', 'ATTACTCCACGAAATA', 'CGGTTAATCTTTAGGG', 'TGACTAGGTTGAGGTG', 'CTGATAGAGCCTATGT', 'GAGTCCGAGGATTCGG', 'ACGTCAATCACTATTC', 'CCGTACTAGGAGTTGC', 'AGTAGTCTCGAGAGCA', 'TTGCCGTCAATGCCAT', 'CACATTTCATGTCCTC', 'ATCATCTTCGCACTCT', 'CGTCCATGTATGAAAC', 'CATTCGCGTTGACGTT', 'ATTGGACGTCTTTCAT', 'ACCGTAAAGCTAACAA', 'ATGGGAGAGGTTACCT', 'TTAGGCATCAGGTTCA', 'CTACCCATCAGGTTCA', 'TGCCCTATCATAACCG', 'CGATTGATCGATGAGG', 'CTCAGAAGTTACGACT', 'TGAGGGAGTCACCTAA', 'AGAATAGAGAACTGTA', 'CCTTACGGTAGAGTGC', 'CAGTCCTAGTTCCACA', 'CGATGGCTCACTATTC', 'TCACGAAAGGAGTAGA', 'GGGCATCAGTTATCGC', 'GCAAACTTCTCAAACG', 'TCCACACAGAGGTACC', 'GGTATTGCATGGTAGG', 'CAACTAGTCACGATGT', 'AAGGCAGGTTACAGAA', 'AACTTTCGTAGTAGTA', 'AGTGTCATCGCCGTGA', 'ATAGACCGT

dict_keys(['LHX3', 'mCherry', 'NRL', 'ASCL3', 'ASCL1', 'LMX1A', 'ASCL4', 'OTX2', 'SNAI2', 'NEUROD1', 'FOXA2', 'NEUROG3', 'NEUROG1', 'ASCL5'])


In [43]:
# is CTTTGCGCATTTGCTT in the adata colums 

print(adata.head())


AttributeError: 'AnnData' object has no attribute 'head'

In [ ]:
# read andata
adata = sc.read_h5ad('./tempdata/GSE107185Anndata_concatinated.h5ad')

# look at headers 
print(adata.obs.head())
print(adata.var.head())
